In [1]:
import requests
import json

# Define admin API URLs for Alice and Bob
CENTRAL_PARTY_ADMIN_URL = "http://localhost:8001"
FL_CLIENT_ADMIN_URL = "http://localhost:8101"


def create_invitation(admin_url):
    """Create a new invitation."""
    response = requests.post(f"{admin_url}/connections/create-invitation")
    return response.json()


def receive_invitation(admin_url, invitation):
    """Receive an invitation."""
    response = requests.post(
        f"{admin_url}/connections/receive-invitation", json=invitation
    )
    return response.json()


def send_message(admin_url, connection_id, message):
    """Send a message to a connection."""
    response = requests.post(
        f"{admin_url}/connections/{connection_id}/send-message",
        json={"content": message},
    )
    return response.json()


def get_messages(admin_url, connection_id):
    """Get all messages for a connection."""
    response = requests.get(f"{admin_url}/connections/{connection_id}/messages")
    return response.json()


def get_connection(admin_url, connection_id):
    """Get the connection details."""
    response = requests.get(f"{admin_url}/connections/{connection_id}")
    return response.json()

In [2]:
print("Central Party: Creating invitation...")
central_party_invitation = create_invitation(CENTRAL_PARTY_ADMIN_URL)
print("Central Party's invitation:", json.dumps(central_party_invitation, indent=2))

Central Party: Creating invitation...
Central Party's invitation: {
  "connection_id": "54bc76d3-a9d3-48fa-839e-2c9b19e88d1d",
  "invitation": {
    "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation",
    "@id": "8a84544d-b477-4acc-a66d-d27eb9bbcf9d",
    "label": "Central Party Agent",
    "recipientKeys": [
      "5sZQwJKnH7q9eZdw6NqtePSjCjGgFASykFZg2at1ZxN3"
    ],
    "serviceEndpoint": "http://central-party-agent:8000"
  },
  "invitation_url": "http://central-party-agent:8000?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiOGE4NDU0NGQtYjQ3Ny00YWNjLWE2NmQtZDI3ZWI5YmJjZjlkIiwgImxhYmVsIjogIkNlbnRyYWwgUGFydHkgQWdlbnQiLCAicmVjaXBpZW50S2V5cyI6IFsiNXNaUXdKS25IN3E5ZVpkdzZOcXRlUFNqQ2pHZ0ZBU3lrRlpnMmF0MVp4TjMiXSwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vY2VudHJhbC1wYXJ0eS1hZ2VudDo4MDAwIn0="
}


In [3]:
print("FL Client: Receiving invitation...")
fl_client_connection = receive_invitation(
    FL_CLIENT_ADMIN_URL, central_party_invitation["invitation"]
)
print("FL Client's connection:", json.dumps(fl_client_connection, indent=2))

FL Client: Receiving invitation...
FL Client's connection: {
  "their_label": "Central Party Agent",
  "accept": "auto",
  "my_did": "DFGjBSy79tN9AS25Y6iir2",
  "invitation_mode": "once",
  "updated_at": "2024-07-06 07:57:45.568790Z",
  "routing_state": "none",
  "connection_id": "6566c00c-6b2e-4aad-a022-afb519c36795",
  "their_role": "inviter",
  "invitation_key": "5sZQwJKnH7q9eZdw6NqtePSjCjGgFASykFZg2at1ZxN3",
  "invitation_msg_id": "8a84544d-b477-4acc-a66d-d27eb9bbcf9d",
  "request_id": "1f223821-580a-4262-9c8e-1413746c1eb8",
  "connection_protocol": "connections/1.0",
  "rfc23_state": "request-sent",
  "state": "request",
  "created_at": "2024-07-06 07:57:45.566234Z"
}


In [4]:
fl_client_connection_id = fl_client_connection["connection_id"]
fl_client_connection = get_connection(FL_CLIENT_ADMIN_URL, fl_client_connection_id)
print(
    "FL Client's connection status:",
    json.dumps(fl_client_connection["state"], indent=2),
)

central_party_connection_id = central_party_invitation["connection_id"]
central_party_connection = get_connection(
    CENTRAL_PARTY_ADMIN_URL, central_party_connection_id
)
print(
    "Central Party's connection status:",
    json.dumps(central_party_connection["state"], indent=2),
)

FL Client's connection status: "request"
Central Party's connection status: "response"


In [5]:
model_params = {"coef": None, "intercept": None}
json_str = json.dumps(model_params)
json_bytes = json_str.encode("utf-8")
hex_str = json_bytes.hex()

In [6]:
print("Initiate FL training run by sending initial weights to client.")
message_response = send_message(
    CENTRAL_PARTY_ADMIN_URL, central_party_connection_id, hex_str
)
print("Message response:", json.dumps(message_response, indent=2))

Initiate FL training run by sending initial weights to client.
Message response: {}
